In [1]:
import os
import numpy as np
from typing import List, Optional, Sequence, Tuple
from PIL import Image

import torch
from torch.utils.data import Dataset, DataLoader

import torch.nn as nn
from torchvision import models
import torchvision.transforms as T
from torch.nn.functional import relu
from torchmetrics.classification import PrecisionRecallCurve
from torch.amp import autocast, GradScaler

from sklearn.model_selection import train_test_split

# Для более сложных аугментаций рекомендуется использовать библиотеку Albumentations
import albumentations as A
from albumentations import Compose, VerticalFlip, RandomRotate90, Affine
from albumentations.pytorch import ToTensorV2

# позволяет удобно отображать прогресс выполнения циклов 
# и других длительных операций прямо в консоли или Jupyter Notebook
from tqdm import tqdm


In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("quadeer15sh/augmented-forest-segmentation")

print("Path to dataset files:", path)

Path to dataset files: /home/pampa89d/.cache/kagglehub/datasets/quadeer15sh/augmented-forest-segmentation/versions/2


In [3]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# ускорить подбор оптимальных алгоритмов свёрток
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.enabled = True

In [4]:
class UNet(nn.Module):
    def __init__(self, n_class):
        super().__init__()

        # ENCODER
        # In the encoder, convolutional layers with the Conv2d function are used to extract features from the input image. 
        # Each block in the encoder consists of two convolutional layers followed by a max-pooling layer, with the exception of the last block which does not include a max-pooling layer.
        # -------
        # input: 572x572x3
        self.e11 = nn.Conv2d(3, 64, kernel_size=3, padding=1) # output: 570x570x64
        self.e12 = nn.Conv2d(64, 64, kernel_size=3, padding=1) # output: 568x568x64
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2) # output: 284x284x64

        # input: 284x284x64
        self.e21 = nn.Conv2d(64, 128, kernel_size=3, padding=1) # output: 282x282x128
        self.e22 = nn.Conv2d(128, 128, kernel_size=3, padding=1) # output: 280x280x128
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2) # output: 140x140x128

        # input: 140x140x128
        self.e31 = nn.Conv2d(128, 256, kernel_size=3, padding=1) # output: 138x138x256
        self.e32 = nn.Conv2d(256, 256, kernel_size=3, padding=1) # output: 136x136x256
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2) # output: 68x68x256

        # input: 68x68x256
        self.e41 = nn.Conv2d(256, 512, kernel_size=3, padding=1) # output: 66x66x512
        self.e42 = nn.Conv2d(512, 512, kernel_size=3, padding=1) # output: 64x64x512
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2) # output: 32x32x512

        # input: 32x32x512
        self.e51 = nn.Conv2d(512, 1024, kernel_size=3, padding=1) # output: 30x30x1024
        self.e52 = nn.Conv2d(1024, 1024, kernel_size=3, padding=1) # output: 28x28x1024

        # DECODER
        self.upconv1 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.d11 = nn.Conv2d(1024, 512, kernel_size=3, padding=1)
        self.d12 = nn.Conv2d(512, 512, kernel_size=3, padding=1)

        self.upconv2 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.d21 = nn.Conv2d(512, 256, kernel_size=3, padding=1)
        self.d22 = nn.Conv2d(256, 256, kernel_size=3, padding=1)

        self.upconv3 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.d31 = nn.Conv2d(256, 128, kernel_size=3, padding=1)
        self.d32 = nn.Conv2d(128, 128, kernel_size=3, padding=1)

        self.upconv4 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.d41 = nn.Conv2d(128, 64, kernel_size=3, padding=1)
        self.d42 = nn.Conv2d(64, 64, kernel_size=3, padding=1)

        # Output layer
        self.outconv = nn.Conv2d(64, n_class, kernel_size=1)

    # определяет последовательность прохождения данных через слои U-Net
    def forward(self, x):
        # Encoder
        x1 = self.e12(self.e11(x))
        p1 = self.pool1(x1)
        x2 = self.e22(self.e21(p1))
        p2 = self.pool2(x2)
        x3 = self.e32(self.e31(p2))
        p3 = self.pool3(x3)
        x4 = self.e42(self.e41(p3))
        p4 = self.pool4(x4)
        x5 = self.e52(self.e51(p4))

        # Decoder
        u1 = self.upconv1(x5)
        c1 = torch.cat([u1, x4], dim=1)
        d1 = self.d12(self.d11(c1))

        u2 = self.upconv2(d1)
        c2 = torch.cat([u2, x3], dim=1)
        d2 = self.d22(self.d21(c2))

        u3 = self.upconv3(d2)
        c3 = torch.cat([u3, x2], dim=1)
        d3 = self.d32(self.d31(c3))

        u4 = self.upconv4(d3)
        c4 = torch.cat([u4, x1], dim=1)
        d4 = self.d42(self.d41(c4))

        out = self.outconv(d4)
        return out

# 1 класс - объект, 0 - пусто
model = UNet(n_class=1)
model.to(device=DEVICE, memory_format=torch.channels_last)

UNet(
  (e11): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (e12): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (e21): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (e22): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (e31): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (e32): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (e41): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (e42): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (e51): Conv2d(512, 1024, kernel_size=(

# 1. Подготовка датасета

In [5]:
''' Обычно рекомендуется хранить изображения и соответствующие маски в отдельных папках 
с одинаковыми именами файлов.

dataset/
  images/
    img1.png
    img2.png
    ...
  masks/
    img1.png
    img2.png
    ...
    
Требования:
- Все изображения и маски должны быть одинакового размера.
- Маски обычно бинарные (0 — фон, 1 или 255 — объект) или многоклассовые.
- Цветовое пространство изображений — обычно RGB, масок — одноканальное.

'''

' Обычно рекомендуется хранить изображения и соответствующие маски в отдельных папках \nс одинаковыми именами файлов.\n\ndataset/\n  images/\n    img1.png\n    img2.png\n    ...\n  masks/\n    img1.png\n    img2.png\n    ...\n    \nТребования:\n- Все изображения и маски должны быть одинакового размера.\n- Маски обычно бинарные (0 — фон, 1 или 255 — объект) или многоклассовые.\n- Цветовое пространство изображений — обычно RGB, масок — одноканальное.\n\n'

# 2. Предобработка и загрузка данных

In [6]:
'''
- Аугментации: Легкие трансформации (повороты, флипы, масштабирование) улучшают обобщаемость модели.

- Класс Dataset: В PyTorch или tf.data (TensorFlow) реализуйте пользовательский загрузчик, 
    который синхронно загружает изображения и соответствующие маски,
    преобразует их в тензоры, нормализует и, при необходимости, применяет аугментации.
'''

'\n- Аугментации: Легкие трансформации (повороты, флипы, масштабирование) улучшают обобщаемость модели.\n\n- Класс Dataset: В PyTorch или tf.data (TensorFlow) реализуйте пользовательский загрузчик, \n    который синхронно загружает изображения и соответствующие маски,\n    преобразует их в тензоры, нормализует и, при необходимости, применяет аугментации.\n'

In [ ]:
class CustomDataset(Dataset):
    """
    Кастомный датасет для задач сегментации лесных изображений.
    
    Пара «изображение-маска» подаётся синхронно и может проходить
    через общие аугментации (Albumentations или иные).
    
    Параметры
    ---------
    root_img : str
        Путь к директории с RGB-изображениями.
    root_msk : str
        Путь к директории с масками (одноканальные PNG/TIFF).
    files : Sequence[str]
        Список имён файлов (без пути), которые будут использоваться
        в этом датасете (например, train или val-список).
    aug : Optional[albumentations.core.composition.BaseCompose]
        Пайплайн аугментаций Albumentations. Если None — аугментации
        не применяются.
    """

    def __init__(
            self,
            image_dir: str,
            mask_dir: str,
            image_files: Sequence[str],
            mask_files: Sequence[str],
            transform: Optional[A.Compose]=None):
        self.image_dir   = image_dir
        self.mask_dir    = mask_dir
        self.image_files = list(image_files)
        self.mask_files  = list(mask_files)
        self.transform   = transform

    def __len__(self) -> int:
        return len(self.image_files)

    def __getitem__(self, idx: int):
    img_name = self.image_files[idx]
    mask_name = self.mask_files[idx]
    img_path = os.path.join(self.image, img_name)
    msk_path = os.path.join(self.mask_dir, mask_name)

    # 1. Load as NumPy arrays
    img_np  = np.array(Image.open(img_path).convert("RGB"), dtype=np.uint8)
    mask_np = np.array(Image.open(msk_path).convert("L"),  dtype=np.uint8)

    # 2. Apply augmentations (if any)
    if self.transform is not None:
        augmented = self.transform(image=img_np, mask=mask_np)
        img_aug, mask_aug = augmented["image"], augmented["mask"]
        # Albumentations ToTensorV2 gives torch.Tensor; other transforms give numpy.ndarray
        if isinstance(img_aug, np.ndarray):
            img_tensor = torch.from_numpy(img_aug).permute(2,0,1).float() / 255.0
        else:
            img_tensor = img_aug.float() / 255.0

        if isinstance(mask_aug, np.ndarray):
            mask_tensor = torch.from_numpy(mask_aug).unsqueeze(0).float() / 255.0
        else:
            mask_tensor = mask_aug.float() / 255.0
            if mask_tensor.ndim == 2:
                mask_tensor = mask_tensor.unsqueeze(0)

    else:
        # No augmentations: convert both arrays to tensors
        img_tensor = torch.from_numpy(img_np).permute(2,0,1).float() / 255.0
        mask_tensor = torch.from_numpy(mask_np).unsqueeze(0).float() / 255.0

    return img_tensor, mask_tensor

In [8]:
# сложные аугментаций 
transform = Compose([
    VerticalFlip(p=0.5),
    RandomRotate90(p=0.5),
    Affine(translate_percent=0.1, scale=(0.9,1.1), rotate=15, p=0.5),
])

In [9]:
# В Python и других программах путь с ~ не раскрывается автоматически, если передаётся как строка. 
# Его нужно явно преобразовать
image_dir=os.path.expanduser('~/.cache/kagglehub/datasets/quadeer15sh/augmented-forest-segmentation/versions/2/Forest_Segmented/images')
mask_dir=os.path.expanduser('~/.cache/kagglehub/datasets/quadeer15sh/augmented-forest-segmentation/versions/2/Forest_Segmented/masks')

images = sorted(os.listdir(image_dir))
masks = sorted(os.listdir(mask_dir))

train_imgs, val_imgs, train_masks, val_masks = train_test_split(
                                            images, masks, test_size=0.2, random_state=42)


In [10]:
train_dataset = CustomDataset(
    image_dir=image_dir,
    mask_dir=mask_dir,
    image_files=train_imgs,
    mask_files=train_masks,
    transform=transform
)
val_dataset = CustomDataset(
    image_dir=image_dir,
    mask_dir=mask_dir,
    image_files=val_imgs,
    mask_files=val_masks,
    transform=transform
)

train_loader = DataLoader(train_dataset, 
                          batch_size=8,
                          shuffle=True,
                          num_workers=0,
                          drop_last=False,
                          pin_memory=False)
valid_loader = DataLoader(val_dataset, 
                        batch_size=8,
                        shuffle=True,
                        num_workers=0,
                        drop_last=False,
                        pin_memory=False)

In [11]:
for images, masks in train_loader:
    print(images.shape, masks.shape)  # Например: torch.Size([8, 3, 256, 256]), torch.Size([8, 1, 256, 256])
    break

torch.Size([8, 3, 256, 256]) torch.Size([8, 1, 256, 256])


In [12]:
for images, masks in valid_loader:
    print(images.shape, masks.shape)  # Например: torch.Size([8, 3, 256, 256]), torch.Size([8, 1, 256, 256])
    break

torch.Size([8, 3, 256, 256]) torch.Size([8, 1, 256, 256])


# функции метрик

In [13]:
def calculate_iou(preds: torch.Tensor, masks: torch.Tensor, threshold: float = 0.5):
    preds = (torch.sigmoid(preds) > threshold).float()
    assert preds.shape == masks.shape, f"Shape mismatch {preds.shape} vs {masks.shape}"
    intersection = (preds * masks).sum(dim=(1,2,3))
    union        = ((preds + masks) > 0).float().sum(dim=(1,2,3))
    return ((intersection + 1e-6) / (union + 1e-6)).mean().item()

def pixel_accuracy(preds: torch.Tensor, masks: torch.Tensor, threshold: float = 0.5):
    preds_bin = (torch.sigmoid(preds) > threshold).long()
    masks_bin = (masks > threshold).long()
    correct = (preds_bin == masks_bin).sum().item()
    total   = masks_bin.numel()
    return correct / total

# масштабирование градиентов AMP
scaler = GradScaler()

# обучение

In [14]:
def save_checkpoint(state: dict, checkpoint_dir: str, epoch: int):
    """
    state: {
      'epoch': текущий номер эпохи (int),
      'model_state': model.state_dict(),
      'optimizer_state': optimizer.state_dict(),
      'scaler_state': scaler.state_dict()  # если используете GradScaler
    }
    checkpoint_dir: путь к директории для чекпойнтов
    epoch: номер эпохи (используется для имени файла)
    """
    os.makedirs(checkpoint_dir, exist_ok=True)
    filename = os.path.join(checkpoint_dir, f'checkpoint_epoch_{epoch:03d}.pth')
    torch.save(state, filename)
    print(f'Checkpoint saved: {filename}')

In [17]:
checkpoint_dir = '../checkpoints'

def fit(model=model, train_loader=train_loader, valid_loader=valid_loader, 
        criterion=torch.nn.Module, optimizer=torch.optim.Optimizer, scaler=scaler, n_epochs=1):
    history = {'train_loss': [], 'val_loss': [], 'train_iou': [], 'val_iou': []}

    for epoch in range(n_epochs):
        # --- TRAIN ---
        model.train()
        running_loss = 0.0
        for imgs, msks in tqdm(train_loader, desc=f"Epoch {epoch+1}/{n_epochs} Train"):
            imgs, msks = imgs.to(DEVICE), msks.to(DEVICE)

            optimizer.zero_grad()
            with torch.amp.autocast('cuda'):
                outputs = model(imgs)
                loss = criterion(outputs, msks)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)
        history['train_loss'].append(avg_train_loss)

        # --- VALID ---
        model.eval()
        val_loss = 0.0
        val_iou  = 0.0
        with torch.no_grad():
            for imgs, msks in tqdm(valid_loader, desc=f"Epoch {epoch+1}/{n_epochs} Valid"):
                imgs, msks = imgs.to(DEVICE), msks.to(DEVICE)
                outputs = model(imgs)
                loss = criterion(outputs, msks)
                val_loss += loss.item()
                val_iou  += calculate_iou(outputs, msks)

        avg_val_loss = val_loss / len(valid_loader)
        avg_val_iou  = val_iou  / len(valid_loader)
        history['val_loss'].append(avg_val_loss)
        history['val_iou'].append(avg_val_iou)

        print(f"Epoch {epoch+1}: Train Loss={avg_train_loss:.4f}, Val Loss={avg_val_loss:.4f}, Val IoU={avg_val_iou:.4f}")

    # сохраняем чекпойнт по окончании эпохи
    save_checkpoint({
        'epoch': epoch + 1,
        'model_state': model.state_dict(),
        'optimizer_state': optimizer.state_dict(),
        'scaler_state': scaler.state_dict(),           # если используете AMP
    }, checkpoint_dir, epoch + 1)

    return history

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.BCELoss()

In [19]:
history = fit(model=model, n_epochs=1, optimizer=optimizer, 
              train_loader=train_loader, valid_loader=valid_loader, 
              criterion=criterion)

Epoch 1/1 Valid: 100%|██████████| 128/128 [00:37<00:00,  3.44it/s]


Epoch 1: Train Loss=nan, Val Loss=732.1196, Val IoU=0.0087
Checkpoint saved: ../checkpoints/checkpoint_epoch_001.pth


In [ ]:
history

{'train_loss': [nan],
 'val_loss': [-4696552.783203125],
 'train_iou': [],
 'val_iou': [140.76532119512558]}